# Surveil — Weapon Detector Training
Trains YOLO11n on real 1920×1080 CCTV footage (Pascal VOC XML annotations).

**Dataset:** `frankmurphy24/cctv-weapon-detector` (Images/ folder)  
**Classes:** `gun` (Handgun), `rifle` (Short_rifle), `knife` (Knife)  
**Output:** `/kaggle/working/runs/weapon/train/weights/best.pt`

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:  # cap at 5 per dir — Images/ has thousands of files
        print(os.path.join(dirname, filename))
    if filenames:
        print(f'  ... ({len(filenames)} files total in {dirname})')
        break  # stop after first dir with files to avoid flooding output

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics -q

In [ ]:
import random
import shutil
import xml.etree.ElementTree as ET
from pathlib import Path

# ---------------------------------------------------------------------------
# Class mapping — normalise all XML name variants to 3 YOLO classes
# ---------------------------------------------------------------------------

CLASS_MAP: dict[str, tuple[int, str]] = {
    # gun (index 0)
    "handgun":    (0, "gun"),
    "Handgun":    (0, "gun"),
    "pistol":     (0, "gun"),
    "Pistol":     (0, "gun"),
    # rifle (index 1)
    "rifle":      (1, "rifle"),
    "Rifle":      (1, "rifle"),
    "short_rifle":(1, "rifle"),
    "Short_rifle":(1, "rifle"),
    # knife (index 2)
    "knife":      (2, "knife"),
    "Knife":      (2, "knife"),
    "machete":    (2, "knife"),
}

CLASSES = ["gun", "rifle", "knife"]

# ---------------------------------------------------------------------------
# Pascal VOC XML → YOLO txt
# ---------------------------------------------------------------------------

def voc_to_yolo(xml_path: Path) -> list[str]:
    """Parse a Pascal VOC XML and return normalised YOLO label lines.
    Returns [] for negative frames (no annotated objects).
    """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
    except ET.ParseError:
        return []

    size = root.find("size")
    if size is None:
        return []
    w = float(size.findtext("width") or 0)
    h = float(size.findtext("height") or 0)
    if w == 0 or h == 0:
        return []

    lines = []
    for obj in root.findall("object"):
        name = obj.findtext("name", "").strip()
        if name not in CLASS_MAP:
            continue
        cls_idx, _ = CLASS_MAP[name]

        bndbox = obj.find("bndbox")
        if bndbox is None:
            continue
        xmin = float(bndbox.findtext("xmin") or 0)
        ymin = float(bndbox.findtext("ymin") or 0)
        xmax = float(bndbox.findtext("xmax") or 0)
        ymax = float(bndbox.findtext("ymax") or 0)

        xmin, xmax = max(0, xmin), min(w, xmax)
        ymin, ymax = max(0, ymin), min(h, ymax)
        if xmax <= xmin or ymax <= ymin:
            continue

        cx = ((xmin + xmax) / 2) / w
        cy = ((ymin + ymax) / 2) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h
        lines.append(f"{cls_idx} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    return lines

# ---------------------------------------------------------------------------
# Dataset assembly
# ---------------------------------------------------------------------------

def collect_pairs(src_dir: Path) -> list[tuple[Path, Path]]:
    """Return (jpg_path, xml_path) pairs where both files exist."""
    pairs = []
    for xml_path in sorted(src_dir.glob("*.xml")):
        jpg_path = xml_path.with_suffix(".jpg")
        if jpg_path.exists():
            pairs.append((jpg_path, xml_path))
    print(f"  {src_dir.name}: {len(pairs)} image/annotation pairs found")
    return pairs


def convert_and_copy(
    pairs: list[tuple[Path, Path]],
    img_out_dir: Path,
    lbl_out_dir: Path,
) -> dict[str, int]:
    """Convert VOC XMLs to YOLO txts and copy images. Returns class counts."""
    img_out_dir.mkdir(parents=True, exist_ok=True)
    lbl_out_dir.mkdir(parents=True, exist_ok=True)

    counts = {"pos": 0, "neg": 0, **{c: 0 for c in CLASSES}}

    for jpg_path, xml_path in pairs:
        lines = voc_to_yolo(xml_path)
        stem = f"{xml_path.parent.name}_{xml_path.stem}"
        shutil.copy(jpg_path, img_out_dir / f"{stem}.jpg")
        (lbl_out_dir / f"{stem}.txt").write_text("\n".join(lines))

        if lines:
            counts["pos"] += 1
            for line in lines:
                counts[CLASSES[int(line.split()[0])]] += 1
        else:
            counts["neg"] += 1

    return counts


def write_yaml(out_dir: Path, val_ratio: float = 0.1) -> Path:
    """Write YOLO dataset YAML with a random 90/10 train/val split."""
    img_dir = out_dir / "images" / "all"
    all_imgs = list(img_dir.glob("*.jpg"))
    random.shuffle(all_imgs)

    n_val = max(1, int(len(all_imgs) * val_ratio))
    val_imgs   = all_imgs[:n_val]
    train_imgs = all_imgs[n_val:]

    for split_name, split_imgs in [("train", train_imgs), ("val", val_imgs)]:
        split_img_dir = out_dir / "images" / split_name
        split_lbl_dir = out_dir / "labels" / split_name
        split_img_dir.mkdir(parents=True, exist_ok=True)
        split_lbl_dir.mkdir(parents=True, exist_ok=True)
        for img in split_imgs:
            lbl = (out_dir / "labels" / "all" / img.name).with_suffix(".txt")
            dst_img = split_img_dir / img.name
            dst_lbl = split_lbl_dir / img.with_suffix(".txt").name
            if not dst_img.exists():
                shutil.copy(img, dst_img)
            if not dst_lbl.exists() and lbl.exists():
                shutil.copy(lbl, dst_lbl)

    yaml_path = out_dir / "weapon.yaml"
    yaml_path.write_text(f"""\
path: {out_dir.resolve()}
train: images/train
val:   images/val

nc: {len(CLASSES)}
names: {CLASSES}
""")
    print(f"  YAML: {yaml_path}")
    print(f"  Train: {len(train_imgs)} images")
    print(f"  Val:   {len(val_imgs)} images")
    return yaml_path

print("Functions defined.")

In [ ]:
# ---------------------------------------------------------------------------
# Paths — Kaggle read-only input on the left, preserved output on the right
# ---------------------------------------------------------------------------

IMAGES_DIR = Path("/kaggle/input/models/frankmurphy24/cctv-weapon-detector/other/default/1/Images")
OUT_DIR    = Path("/kaggle/working/weapon_dataset")
RUNS_DIR   = "/kaggle/working/runs/weapon"   # best.pt ends up here
VAL_RATIO  = 0.1

assert IMAGES_DIR.exists(), f"Images directory not found: {IMAGES_DIR}\nCheck that the dataset is attached to this notebook."
print(f"Images dir: {IMAGES_DIR}")
print(f"Output dir: {OUT_DIR}")
print(f"XML count:  {len(list(IMAGES_DIR.glob('*.xml')))}")
print(f"JPG count:  {len(list(IMAGES_DIR.glob('*.jpg')))}")

In [ ]:
# ---------------------------------------------------------------------------
# Step 1 — Collect paired image/annotation files
# ---------------------------------------------------------------------------

print("=== STEP 1: Collect image/annotation pairs ===")
pairs = collect_pairs(IMAGES_DIR)
print(f"  Total: {len(pairs)} pairs")

In [ ]:
# ---------------------------------------------------------------------------
# Step 2 — Convert Pascal VOC XML → YOLO txt, copy images to working dir
# ---------------------------------------------------------------------------

print("=== STEP 2: Convert Pascal VOC → YOLO ===")
out_img_dir = OUT_DIR / "images" / "all"
out_lbl_dir = OUT_DIR / "labels" / "all"
counts = convert_and_copy(pairs, out_img_dir, out_lbl_dir)
print(f"  Positive frames (with objects): {counts['pos']}")
print(f"  Negative frames (no objects):   {counts['neg']}")
for cls in CLASSES:
    print(f"    {cls}: {counts[cls]} instances")

In [ ]:
# ---------------------------------------------------------------------------
# Step 3 — Build 90/10 train/val split and write YOLO dataset YAML
# ---------------------------------------------------------------------------

print("=== STEP 3: Build train/val split ===")
yaml_path = write_yaml(OUT_DIR, VAL_RATIO)

In [ ]:
# ---------------------------------------------------------------------------
# Step 4 — Train YOLO11n (150 epochs, 640px, T4 GPU)
# best.pt saved to /kaggle/working/runs/weapon/train/weights/best.pt
# ---------------------------------------------------------------------------

from ultralytics import YOLO

print("=== STEP 4: Train YOLO11n ===")
model = YOLO("yolo11n.pt")
model.train(
    data=str(yaml_path),
    epochs=150,
    imgsz=640,
    batch=64,
    device=0,             # Kaggle T4 GPU
    project=RUNS_DIR,
    name="train",
    exist_ok=True,
    # Augmentation tuned for CCTV: slight rotation, no vertical flip, colour jitter
    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.5,
    degrees=5,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
)

best = Path(RUNS_DIR) / "train" / "weights" / "best.pt"
print(f"\n{'='*60}")
print(f"Training complete!")
print(f"Best weights: {best}")
print(f"\nDownload best.pt from the Output tab, then:")
print(f"  cp best.pt backend/models/weapon.pt")
print(f"{'='*60}")